# 目的
あるクラウドファンディングが成功するか(state)を事前に予測するモデルを構築する。

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error # 回帰問題における性能評価に関する関数
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix # 回帰問題における性能評価に関する関数

# データの読み込み

## データの扱いについて
| Variables | 変数 | 使う? | 変数の扱い |
| ---- | ---- | ---- |  ---- |
| ID | internal kickstarter id | - | - |
| name | プロジェクト名 | - | - |
| category | カテゴリー | ○ | ダミー変数 |
| main_category | category of campaign | ○ | ダミー変数 |
| currency | 通貨 | ○ | ダミー変数 |
| deadline | 期限 | ○ (開始日 - 期限 の募集機関で使ってみる) | 数値(日数) |
| goal | 目標金額 | ○ | 数値(金額) |
| launched | 開始日 | ○ (開始日 - 期限 の募集機関で使ってみる) | 数値(日数) |
| pledged | 集まった金額 | - (開始時にはわからないので使わない) | - |
| state | 成否 | ※目的変数 | fiailled: 0, successful: 1, Other: 捨てる(Pandasで) |

↓必要な項目を抜き出して読み込み、データの整形を行う。

In [37]:
ks_projects_data = pd.read_csv("ks-projects-201801.csv")[['category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'state']]

# state が failed と successful 以外の行を削除
ks_projects_data2 = ks_projects_data[(ks_projects_data['state'] == 'failed') | (ks_projects_data['state'] == 'successful')]

# 欠損地がないことを確認
#ks_projects_data2.info()

display(ks_projects_data2.head())
#print(type(ks_projects_data2.loc[1, 'launched']))
#print(datetime.datetime.strptime(ks_projects_data2['launched'], '%Y-%m-%d %H:%M:%S'))

# deadline - launched で募集期間を算出する
duration = []
ks_projects_data2['durationi'] = 0

#for i in range(len(ks_projects_data2.index)):
#    launched_day = datetime.datetime.strptime(ks_projects_data2.loc[i, 'launched'], '%Y-%m-%d %H:%M:%S')
#    deadline_day = datetime.datetime.strptime(ks_projects_data2.loc[i, 'deadline'], '%Y-%m-%d')
#    duration.append((deadline_day - launched_day).days)
#    ks_projects_data2.loc[i, 'duration'] = (deadline_day - launched_day).days
    
    
#ks_projects_data2['duration'] = duration
display(ks_projects_data2.head())


#launched_day = datetime.datetime.strptime(ks_projects_data2.loc[0, 'launched'], '%Y-%m-%d %H:%M:%S')
#deadline_day = datetime.datetime.strptime(ks_projects_data2.loc[0, 'deadline'], '%Y-%m-%d')
#duration = deadline_day - launched_day
#print(duration.days)


#print(type(datetime.datetime.strptime(ks_projects_data2.loc[1, 'launched'], '%Y-%m-%d %H:%M:%S')))




,category,main_category,currency,deadline,goal,launched,state
0,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,failed
1,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,failed
2,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,failed
3,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,failed
5,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,successful


C:\Users\md\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,category,main_category,currency,deadline,goal,launched,state,durationi
0,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,failed,0
1,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,failed,0
2,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,failed,0
3,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,failed,0
5,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,successful,0
